In [ ]:
from csw_search import *

In [ ]:
from owslib.csw import CatalogueServiceWeb

In [ ]:
from datetime import datetime

In [ ]:
def csw_query(csw_service, bbox=None, start=None, stop=None, kw_names=None, crs="urn:ogc:def:crs:OGC:1.3:CRS84"):
    constraints = []
    if kw_names:
        kw = dict(wildCard="*", escapeChar="\\", singleChar="?", propertyname="apiso:AnyText")
        or_filt = fes.Or([fes.PropertyIsLike(literal=("*%s*" % val), **kw) for val in kw_names])
        constraints.append(or_filt)
        
    if all(v is not None for v in [start, stop]): 
        begin, end = fes_date_filter(start, stop)
        constraints.append(begin)
        constraints.append(end)
        
    if bbox:
        bbox_crs = fes.BBox(bbox, crs=crs)
        constraints.append(bbox_crs)
    if len(constraints) >= 2:
        filter_list = [
            fes.And(
                constraints
            )
        ]
    else:
        filter_list = constraints
    get_csw_records(csw_service, filter_list, pagesize=10, maxrecords=1000)

    records = "\n".join(csw_service.records.keys())
    print("Found {} records.\n".format(len(csw_service.records.keys())))
    for key, value in list(csw_service.records.items()):
        print(u"[{}]\n{}\n".format(value.title, key))

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, GeoJSON
global box
from ipydatetime import DatetimePicker
watercolor = basemap_to_tiles(basemaps.Stamen.Watercolor)

from sidecar import Sidecar

import ipyleaflet as L
from ipywidgets import widgets as w
from datetime import datetime
import pandas as pd

import dateutil
import dateutil.parser

lat_label = w.Label()
lon_label = w.Label()
lc = L.LayersControl(position='topright')

endpoint_entry = w.Text(
    value='https://csw.epinux.com',
    placeholder='CSW Catalogue endpoint',
    description='',
    disabled=False
)

datetime_picker_start = DatetimePicker()
datetime_picker_end = DatetimePicker()

def mk_clear_button(target, action_name):
    b = w.Button(description=action_name.replace('_', ' '))
    action = getattr(target, action_name)
    b.on_click(lambda *a: action())
    return b


out = w.Output(layout=w.Layout(width='50%', 
                               height='400px', 
                               overflow_y='scroll'))
m = L.Map(layers=(watercolor, ), center=(74, 378), zoom=5)

draw = L.DrawControl(
    edit=True,
    remove=True,
    circlemarker={},
    marker={},
    circle={},    
    polyline={},
    polygon = {},
    rectangle={'shapeOptions': {}})


feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        coords = kwargs.get('coordinates')
        #lon = coords[1]-360
        #lat = coords[0]
        lat_label.value = str(coords[0]) 
        lon_label.value = str(coords[1]-360)

def handle_click(**kwargs):
    with out:
        print(kwargs)
        
def on_draw_handler(draw, action, geo_json):
    
    # m.remove_layer(draw.last_drawn)
    with out:
        out.clear_output()
        for i in m.layers:
            if type(i) == L.GeoJSON:
                print('ok')
                m.remove_layer(i)
        bounds = geo_json['geometry']['coordinates'][0]
        bounds = [[i[0]-360, i[1]] for i in bounds if i[0]>=180]
        ll = bounds[0]
        ur = bounds[2]
        print(ll,ur)

        corners = [ll, ur]
        bbox = [item for sublist in corners for item in sublist]
        csw = None
        while csw is None:
            try:
                # connect
                endpoint = endpoint_entry.value
                #endpoint = "https://nbs.csw.met.no"
                csw = CatalogueServiceWeb(endpoint, timeout=60)
            except:
                pass
        start = dateutil.parser.parse(str(datetime_picker_start.value))
        end = dateutil.parser.parse(str(datetime_picker_end.value))
        csw_query(csw_service=csw, bbox=bbox, start=start, stop=end, kw_names=None, crs="urn:ogc:def:crs:OGC:1.3:CRS84")
        searchbox = GeoJSON(data=geo_json)
        searchbox.on_click(handle_click)
        m.add_layer(searchbox)
        draw.clear_rectangles()
        

#btns = [] # [mk_clear_button(draw, 'clear_rectangles')]
#btns.append(mk_clear_button(out, 'clear_output'))
clear_output = mk_clear_button(out, 'clear_output')
draw.on_draw(on_draw_handler)
m.add_control(draw)
m.add_control(lc)
m.on_interaction(handle_interaction)

dashboard = w.VBox([w.HBox([w.Label(value="CSW Endpoint:"), endpoint_entry]), 
                    w.HBox([datetime_picker_start, datetime_picker_end]), 
                    w.VBox([clear_output]), 
                    w.HBox([m, out]), w.HBox([lon_label, lat_label])])



dashboard
